In [81]:
import re
import json
import statistics

from pathlib import Path

import numpy as np

In [3]:
def load_original_data(target):
    paths = Path("data/results/").glob(f"{target}*.json")
    results = {}
    for path in paths:
        pattern = f"{target}_(\\d+.\\d+)eps_(\\d+)steps"
        epsilon, steps = re.fullmatch(pattern, path.stem).groups()
        epsilon = float(epsilon)
        if epsilon > 0.07:
            continue
        with open(path, 'r') as f:
            contents = json.load(f)
            results[epsilon] = contents
    return results

In [ ]:
def load_ablation_data(target):
    paths = Path("data/results/").glob(f"ablation_{target}*.json")
    results = {}
    for path in paths:
        pattern = f"ablation_{target}_(\\d+.\\d+)eps_(\\d+)steps"
        epsilon, steps = re.fullmatch(pattern, path.stem).groups()
        epsilon = float(epsilon)
        with open(path, 'r') as f:
            contents = json.load(f)
            results[epsilon] = contents
    return results

In [5]:
load_ablation_data("wind")

{0.05: {'w/o approx': [[-13.877053260803223,
    -15.082160949707031,
    -17.703166961669922,
    -10.44516658782959,
    -14.186493873596191],
   [-12.078812599182129,
    -8.668627738952637,
    -9.741235733032227,
    -7.720380783081055,
    -11.02860164642334]],
  'w/o steps': [[-16.39448356628418,
    -16.7823543548584,
    -20.8136043548584,
    -17.182479858398438,
    -18.22907257080078],
   [-10.290508270263672,
    -10.737956047058105,
    -11.235365867614746,
    -10.037707328796387,
    -11.324209213256836]],
  'unperturbed': [[-2.3612241744995117,
    -1.455011010169983,
    -3.7872977256774902,
    -2.1405563354492188,
    -3.3234262466430664],
   [-0.6483078002929688,
    -1.1738452911376953,
    -1.2550204992294312,
    -0.07598818093538284,
    -1.4091135263442993]]},
 0.03: {'w/o approx': [[-4.303272724151611,
    -4.067123889923096,
    -6.8184590339660645,
    -3.9630863666534424,
    -5.265993595123291],
   [-4.191310405731201,
    -4.891841411590576,
    -5.12547

In [6]:
def match_ablation_and_original(target, offset):
    original = load_original_data(target)
    ablation = load_ablation_data(target)
    combined = {}
    for epsilon in ablation.keys():
        combined[epsilon] = {}
        for attack, values in ablation[epsilon].items():
            combined[epsilon]["ablation "+attack] = values
        length = min(len(v) for v in combined[epsilon].values())
        for attack, values in original[epsilon].items():
            combined[epsilon][attack] = values[offset:(offset+length)]
    return combined

In [48]:
load_original_data("wind")[0.03].keys()

dict_keys(['Ours', 'AdvDM', 'DP-Attacker', 'unperturbed'])

In [88]:
def summarize_ablation(target, offset):
    combined = match_ablation_and_original(target, offset)
    results = {"w/o steps": [], "w/o approx": [], "w/o both": []}
    for epsilon in combined.keys():
        unperturbed = -np.asarray(combined[epsilon]["unperturbed"])
        abl_unperturbed = -np.asarray(combined[epsilon]["ablation unperturbed"])
        assert np.allclose(unperturbed, abl_unperturbed, atol=1e-2, rtol=1e-4), np.max(np.abs(unperturbed - abl_unperturbed))

        our_deviations = np.median(-np.asarray(combined[epsilon]["Ours"]) - unperturbed, axis=-1)
        baseline_deviations = np.median(-np.asarray(combined[epsilon]["DP-Attacker"]) - unperturbed, axis=-1)
        wo_approx_deviations = np.median(-np.asarray(combined[epsilon]["ablation w/o approx"]) - abl_unperturbed, axis=-1)
        wo_steps_deviations = np.median(-np.asarray(combined[epsilon]["ablation w/o steps"]) - abl_unperturbed, axis=-1)
        
        results["w/o both"].extend((baseline_deviations / our_deviations).tolist())
        results["w/o approx"].extend((wo_approx_deviations / our_deviations).tolist())
        results["w/o steps"].extend((wo_steps_deviations / our_deviations).tolist())
    aggregated = {}
    for k, vs in results.items():
        mean = round(sum(vs)/len(vs)*100, 2)
        ci = 1.644850 * statistics.stdev(vs) / len(vs)**0.5
        ci = round(ci*100, 2)
        aggregated[k] = (mean, ci, len(vs))
    return aggregated

In [97]:
wind = summarize_ablation("wind", 0)
temperature = summarize_ablation("temperature", 34)
precipitation = summarize_ablation("precipitation", 68)
wind, temperature, precipitation

({'w/o steps': (89.26, 2.45, 68),
  'w/o approx': (59.29, 4.2, 68),
  'w/o both': (56.0, 2.96, 68)},
 {'w/o steps': (93.07, 1.42, 68),
  'w/o approx': (71.57, 3.12, 68),
  'w/o both': (62.85, 3.27, 68)},
 {'w/o steps': (54.42, 6.5, 64),
  'w/o approx': (33.91, 4.68, 64),
  'w/o both': (18.36, 4.26, 64)})